# 🧪 Docker Compose Jupyter Lab Integration Test

This notebook validates the complete Docker Compose setup for Jupyter Lab, ensuring all components work correctly end-to-end.

## Test Objectives
- ✅ Validate Docker Compose configuration
- ✅ Test Jupyter server connectivity and API
- ✅ Verify data science packages functionality
- ✅ Test volume mounts and file operations
- ✅ Generate comprehensive test reports
- ✅ Save results to scripts/tests/results folder

---

## 1. Import Required Libraries

Import all necessary libraries for testing Docker Compose configuration and Jupyter functionality:

In [ ]:
import subprocess
import os
import json
import sys
from datetime import datetime
from pathlib import Path
import time

# Data science libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from sklearn import datasets

# Test configuration
TEST_CONFIG = {
    "base_url": "http://localhost:8888",
    "token": "datascience-token",
    "test_timestamp": datetime.now().isoformat(),
    "project_root": "/home/jovyan/work",
    "results_dir": "/home/jovyan/work/scripts/tests/results"
}

print("✅ All libraries imported successfully!")
print(f"Test Configuration: {json.dumps(TEST_CONFIG, indent=2)}")
print(f"Current working directory: {os.getcwd()}")
print(f"Python version: {sys.version}")

## 2. Setup Test Environment

Create the directory structure and initialize test tracking:

In [ ]:
# Initialize test tracking
test_results = []
test_counter = 0

def log_test_result(test_name, status, message="", details=None):
    """Log test results with standardized format"""
    global test_counter
    test_counter += 1
    
    result = {
        "test_id": test_counter,
        "timestamp": datetime.now().isoformat(),
        "test_name": test_name,
        "status": status,
        "message": message,
        "details": details or {}
    }
    test_results.append(result)
    
    status_icon = "✅" if status == "PASS" else "❌" if status == "FAIL" else "⚠️"
    print(f"{status_icon} Test {test_counter}: {test_name}")
    if message:
        print(f"   └── {message}")
    return result

# Create test directories
tests_dir = Path(TEST_CONFIG["project_root"]) / "scripts" / "tests"
results_dir = Path(TEST_CONFIG["results_dir"])

tests_dir.mkdir(parents=True, exist_ok=True)
results_dir.mkdir(parents=True, exist_ok=True)

log_test_result("Test Environment Setup", "PASS", f"Created directories: {tests_dir}, {results_dir}")

print(f"\n📁 Test Environment Ready:")
print(f"   Tests directory: {tests_dir}")
print(f"   Results directory: {results_dir}")
print(f"   Current directory: {Path.cwd()}")

## 3. Test Docker Compose Configuration

Validate the Docker Compose setup and container environment:

In [ ]:
# Test 1: Check if we're running in the expected container environment
def test_container_environment():
    """Test if we're running in the correct Docker container"""
    try:
        # Check hostname (should be container ID)
        hostname = os.uname().nodename
        
        # Check if we're running as jovyan user
        user = os.environ.get('USER', 'unknown')
        
        # Check if conda is available (part of datascience-notebook)
        conda_result = subprocess.run(['conda', '--version'], capture_output=True, text=True)
        
        details = {
            "hostname": hostname,
            "user": user,
            "conda_available": conda_result.returncode == 0,
            "conda_version": conda_result.stdout.strip() if conda_result.returncode == 0 else "Not available"
        }
        
        if user == 'jovyan' and conda_result.returncode == 0:
            return log_test_result("Container Environment", "PASS", f"Running as {user} with conda", details)
        else:
            return log_test_result("Container Environment", "FAIL", f"Unexpected environment: user={user}", details)
            
    except Exception as e:
        return log_test_result("Container Environment", "FAIL", f"Error checking environment: {str(e)}")

# Test 2: Verify volume mounts
def test_volume_mounts():
    """Test if all expected volume mounts are accessible"""
    expected_paths = [
        "/home/jovyan/work/notebooks",
        "/home/jovyan/work/data", 
        "/home/jovyan/work/scripts",
        "/home/jovyan/work/outputs"
    ]
    
    accessible_paths = []
    missing_paths = []
    
    for path in expected_paths:
        if Path(path).exists():
            accessible_paths.append(path)
        else:
            missing_paths.append(path)
    
    if not missing_paths:
        return log_test_result("Volume Mounts", "PASS", f"All {len(expected_paths)} paths accessible", 
                             {"accessible_paths": accessible_paths})
    else:
        return log_test_result("Volume Mounts", "FAIL", f"Missing paths: {missing_paths}",
                             {"accessible_paths": accessible_paths, "missing_paths": missing_paths})

# Test 3: Check data science packages
def test_data_science_packages():
    """Test availability and versions of key data science packages"""
    packages_to_test = {
        'pandas': pd,
        'numpy': np, 
        'matplotlib': plt.matplotlib,
        'seaborn': sns,
        'plotly': px.plotly,
        'sklearn': datasets.sklearn
    }
    
    package_info = {}
    failed_packages = []
    
    for name, module in packages_to_test.items():
        try:
            version = getattr(module, '__version__', 'Version not available')
            package_info[name] = version
        except Exception as e:
            failed_packages.append(f"{name}: {str(e)}")
    
    if not failed_packages:
        return log_test_result("Data Science Packages", "PASS", 
                             f"All {len(packages_to_test)} packages available", package_info)
    else:
        return log_test_result("Data Science Packages", "FAIL", 
                             f"Failed packages: {failed_packages}", package_info)

# Run Docker Compose tests
print("🐳 Running Docker Compose Configuration Tests...\n")

test_container_environment()
test_volume_mounts() 
test_data_science_packages()

print(f"\n📊 Docker Tests Completed: {len([r for r in test_results if r['test_name'] in ['Container Environment', 'Volume Mounts', 'Data Science Packages']])} tests run")

## 4. Test Jupyter Notebook Connectivity

Verify Jupyter server functionality and API endpoints:

In [ ]:
# Test 4: Jupyter Server Status
def test_jupyter_server():
    """Test if Jupyter server is running and accessible internally"""
    try:
        # Check if we can access Jupyter internals
        from IPython import get_ipython
        from jupyter_client import find_connection_file
        
        ipython = get_ipython()
        kernel_id = ipython.kernel.ident if ipython and hasattr(ipython, 'kernel') else 'unknown'
        
        # Try to find connection file
        try:
            connection_file = find_connection_file()
            connection_available = True
        except:
            connection_file = "Not found"
            connection_available = False
        
        details = {
            "ipython_available": ipython is not None,
            "kernel_id": str(kernel_id),
            "connection_file": str(connection_file),
            "connection_available": connection_available
        }
        
        if ipython is not None:
            return log_test_result("Jupyter Server Status", "PASS", 
                                 "IPython kernel running", details)
        else:
            return log_test_result("Jupyter Server Status", "FAIL", 
                                 "IPython not available", details)
            
    except Exception as e:
        return log_test_result("Jupyter Server Status", "FAIL", 
                             f"Error checking Jupyter: {str(e)}")

# Test 5: Kernel Functionality  
def test_kernel_functionality():
    """Test kernel execution capabilities"""
    try:
        # Test basic Python execution
        result = eval("2 + 2")
        
        # Test variable assignment and retrieval
        test_var = "jupyter_test_variable"
        globals()[test_var] = "test_value"
        var_test = globals().get(test_var) == "test_value"
        
        # Test import capabilities
        import random
        random_test = random.randint(1, 100)
        
        details = {
            "basic_math": result == 4,
            "variable_assignment": var_test,
            "import_test": isinstance(random_test, int),
            "random_value": random_test
        }
        
        all_tests_passed = all(details.values())
        
        if all_tests_passed:
            return log_test_result("Kernel Functionality", "PASS", 
                                 "All kernel operations working", details)
        else:
            return log_test_result("Kernel Functionality", "FAIL", 
                                 "Some kernel operations failed", details)
            
    except Exception as e:
        return log_test_result("Kernel Functionality", "FAIL", 
                             f"Kernel test error: {str(e)}")

# Test 6: File System Operations
def test_filesystem_operations():
    """Test file system read/write operations"""
    try:
        test_file_path = Path("/home/jovyan/work/notebooks/kernel_test.txt")
        test_content = f"Kernel test file created at {datetime.now().isoformat()}"
        
        # Write test
        with open(test_file_path, 'w') as f:
            f.write(test_content)
        write_success = test_file_path.exists()
        
        # Read test
        if write_success:
            with open(test_file_path, 'r') as f:
                read_content = f.read()
            read_success = read_content == test_content
            
            # Cleanup
            test_file_path.unlink()
            cleanup_success = not test_file_path.exists()
        else:
            read_success = False
            cleanup_success = False
        
        details = {
            "write_operation": write_success,
            "read_operation": read_success, 
            "cleanup_operation": cleanup_success,
            "file_path": str(test_file_path)
        }
        
        if write_success and read_success and cleanup_success:
            return log_test_result("File System Operations", "PASS",
                                 "All file operations successful", details)
        else:
            return log_test_result("File System Operations", "FAIL",
                                 "Some file operations failed", details)
            
    except Exception as e:
        return log_test_result("File System Operations", "FAIL", 
                             f"File system test error: {str(e)}")

# Run Jupyter connectivity tests
print("📡 Running Jupyter Connectivity Tests...\n")

test_jupyter_server()
test_kernel_functionality() 
test_filesystem_operations()

print(f"\n📊 Jupyter Tests Completed: {len([r for r in test_results if r['test_name'] in ['Jupyter Server Status', 'Kernel Functionality', 'File System Operations']])} tests run")

## 5. Run End-to-End Integration Tests

Execute comprehensive tests including data processing and visualization:

In [ ]:
# Test 7: Data Processing Pipeline
def test_data_processing():
    """Test complete data science workflow"""
    try:
        # Create sample dataset
        np.random.seed(42)
        data = {
            'feature_1': np.random.randn(100),
            'feature_2': np.random.randn(100),
            'category': np.random.choice(['A', 'B', 'C'], 100),
            'target': np.random.randint(0, 2, 100)
        }
        df = pd.DataFrame(data)
        
        # Test pandas operations
        df_summary = df.describe()
        df_grouped = df.groupby('category').mean()
        
        # Test numpy operations
        correlation_matrix = np.corrcoef(df['feature_1'], df['feature_2'])
        
        details = {
            "dataframe_shape": df.shape,
            "summary_stats": df_summary.shape,
            "grouped_data": df_grouped.shape,
            "correlation": float(correlation_matrix[0, 1]),
            "data_types": list(df.dtypes.astype(str))
        }
        
        # Validate results
        tests_passed = [
            df.shape == (100, 4),
            df_summary.shape[1] == 3,  # 3 numeric columns
            len(df_grouped) == 3,      # 3 categories
            -1 <= correlation_matrix[0, 1] <= 1  # Valid correlation
        ]
        
        if all(tests_passed):
            return log_test_result("Data Processing Pipeline", "PASS",
                                 "All data operations successful", details)
        else:
            return log_test_result("Data Processing Pipeline", "FAIL",
                                 f"Failed validations: {sum(tests_passed)}/{len(tests_passed)}", details)
            
    except Exception as e:
        return log_test_result("Data Processing Pipeline", "FAIL", 
                             f"Data processing error: {str(e)}")

# Test 8: Visualization Capabilities
def test_visualization():
    """Test matplotlib and plotly visualization"""
    try:
        # Create test data
        x = np.linspace(0, 10, 50)
        y = np.sin(x)
        
        # Test matplotlib
        fig, ax = plt.subplots(figsize=(8, 6))
        ax.plot(x, y, label='sin(x)')
        ax.set_title('Test Plot')
        ax.legend()
        
        # Save matplotlib figure
        output_path = Path("/home/jovyan/work/outputs/figures")
        output_path.mkdir(parents=True, exist_ok=True)
        
        mpl_file = output_path / "test_matplotlib.png"
        plt.savefig(mpl_file, dpi=100, bbox_inches='tight')
        plt.close()
        
        mpl_success = mpl_file.exists()
        
        # Test plotly
        fig_plotly = go.Figure()
        fig_plotly.add_trace(go.Scatter(x=x, y=y, mode='lines', name='sin(x)'))
        fig_plotly.update_layout(title='Test Plotly Plot')
        
        # Save plotly figure
        plotly_file = output_path / "test_plotly.html"
        fig_plotly.write_html(plotly_file)
        
        plotly_success = plotly_file.exists()
        
        details = {
            "matplotlib_file_created": mpl_success,
            "matplotlib_file_path": str(mpl_file),
            "plotly_file_created": plotly_success,
            "plotly_file_path": str(plotly_file),
            "output_directory": str(output_path)
        }
        
        if mpl_success and plotly_success:
            return log_test_result("Visualization Capabilities", "PASS",
                                 "Both matplotlib and plotly working", details)
        else:
            return log_test_result("Visualization Capabilities", "FAIL",
                                 f"Matplotlib: {mpl_success}, Plotly: {plotly_success}", details)
            
    except Exception as e:
        return log_test_result("Visualization Capabilities", "FAIL", 
                             f"Visualization error: {str(e)}")

# Test 9: Machine Learning Workflow
def test_machine_learning():
    """Test basic machine learning workflow"""
    try:
        from sklearn.model_selection import train_test_split
        from sklearn.ensemble import RandomForestClassifier
        from sklearn.metrics import accuracy_score
        
        # Load sample dataset
        iris = datasets.load_iris()
        X, y = iris.data, iris.target
        
        # Split data
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        
        # Train model
        model = RandomForestClassifier(n_estimators=10, random_state=42)
        model.fit(X_train, y_train)
        
        # Make predictions
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        
        # Save model results
        model_output_dir = Path("/home/jovyan/work/outputs/models")
        model_output_dir.mkdir(parents=True, exist_ok=True)
        
        import pickle
        model_file = model_output_dir / "test_model.pkl"
        with open(model_file, 'wb') as f:
            pickle.dump(model, f)
        
        model_saved = model_file.exists()
        
        details = {
            "dataset_shape": X.shape,
            "train_size": X_train.shape[0],
            "test_size": X_test.shape[0],
            "accuracy": float(accuracy),
            "model_saved": model_saved,
            "model_file": str(model_file)
        }
        
        # Validate results
        tests_passed = [
            accuracy > 0.8,  # Reasonable accuracy
            model_saved,     # Model saved successfully
            len(y_pred) == len(y_test)  # Predictions match test set
        ]
        
        if all(tests_passed):
            return log_test_result("Machine Learning Workflow", "PASS",
                                 f"ML pipeline successful (accuracy: {accuracy:.3f})", details)
        else:
            return log_test_result("Machine Learning Workflow", "FAIL",
                                 f"Failed ML tests: {sum(tests_passed)}/{len(tests_passed)}", details)
            
    except Exception as e:
        return log_test_result("Machine Learning Workflow", "FAIL", 
                             f"ML workflow error: {str(e)}")

# Run end-to-end integration tests
print("🔄 Running End-to-End Integration Tests...\n")

test_data_processing()
test_visualization()
test_machine_learning()

print(f"\n📊 Integration Tests Completed: {len([r for r in test_results if r['test_name'] in ['Data Processing Pipeline', 'Visualization Capabilities', 'Machine Learning Workflow']])} tests run")

## 6. Generate Test Reports

Create comprehensive test reports with metrics and analysis:

In [ ]:
# Generate comprehensive test report
def generate_test_summary():
    """Generate summary statistics and analysis"""
    
    # Calculate statistics
    total_tests = len(test_results)
    passed_tests = len([r for r in test_results if r['status'] == 'PASS'])
    failed_tests = len([r for r in test_results if r['status'] == 'FAIL'])
    warning_tests = len([r for r in test_results if r['status'] == 'WARN'])
    
    success_rate = (passed_tests / total_tests) * 100 if total_tests > 0 else 0
    
    # Categorize tests by type
    test_categories = {
        'Docker Environment': ['Container Environment', 'Volume Mounts', 'Data Science Packages'],
        'Jupyter Functionality': ['Jupyter Server Status', 'Kernel Functionality', 'File System Operations'],
        'Data Science Workflow': ['Data Processing Pipeline', 'Visualization Capabilities', 'Machine Learning Workflow']
    }
    
    category_results = {}
    for category, test_names in test_categories.items():
        category_tests = [r for r in test_results if r['test_name'] in test_names]
        category_passed = len([r for r in category_tests if r['status'] == 'PASS'])
        category_total = len(category_tests)
        category_rate = (category_passed / category_total) * 100 if category_total > 0 else 0
        
        category_results[category] = {
            'passed': category_passed,
            'total': category_total,
            'success_rate': category_rate
        }
    
    # Create summary object
    summary = {
        'test_run_info': {
            'timestamp': TEST_CONFIG['test_timestamp'],
            'total_tests': total_tests,
            'passed': passed_tests,
            'failed': failed_tests,
            'warnings': warning_tests,
            'success_rate': success_rate
        },
        'category_breakdown': category_results,
        'detailed_results': test_results
    }
    
    return summary

# Display test summary
summary = generate_test_summary()

print("📊 TEST SUMMARY REPORT")
print("=" * 50)
print(f"Total Tests Run: {summary['test_run_info']['total_tests']}")
print(f"✅ Passed: {summary['test_run_info']['passed']}")
print(f"❌ Failed: {summary['test_run_info']['failed']}")
print(f"⚠️  Warnings: {summary['test_run_info']['warnings']}")
print(f"🎯 Success Rate: {summary['test_run_info']['success_rate']:.1f}%")

print(f"\n📈 CATEGORY BREAKDOWN")
print("-" * 30)
for category, results in summary['category_breakdown'].items():
    status_icon = "✅" if results['success_rate'] == 100 else "⚠️" if results['success_rate'] >= 80 else "❌"
    print(f"{status_icon} {category}: {results['passed']}/{results['total']} ({results['success_rate']:.1f}%)")

print(f"\n📋 DETAILED RESULTS")
print("-" * 30)
for result in test_results:
    status_icon = "✅" if result['status'] == 'PASS' else "❌" if result['status'] == 'FAIL' else "⚠️"
    print(f"{status_icon} Test {result['test_id']}: {result['test_name']}")
    if result['message']:
        print(f"   └── {result['message']}")

# Visual summary with matplotlib
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Test results pie chart
labels = ['Passed', 'Failed', 'Warnings']
sizes = [summary['test_run_info']['passed'], 
         summary['test_run_info']['failed'], 
         summary['test_run_info']['warnings']]
colors = ['#2ecc71', '#e74c3c', '#f39c12']

# Only show non-zero categories
non_zero_data = [(label, size, color) for label, size, color in zip(labels, sizes, colors) if size > 0]
if non_zero_data:
    labels, sizes, colors = zip(*non_zero_data)
    ax1.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
ax1.set_title('Overall Test Results')

# Category success rates bar chart
categories = list(summary['category_breakdown'].keys())
success_rates = [summary['category_breakdown'][cat]['success_rate'] for cat in categories]

bars = ax2.bar(range(len(categories)), success_rates, color=['#2ecc71' if rate == 100 else '#f39c12' if rate >= 80 else '#e74c3c' for rate in success_rates])
ax2.set_xlabel('Test Categories')
ax2.set_ylabel('Success Rate (%)')
ax2.set_title('Success Rate by Category')
ax2.set_xticks(range(len(categories)))
ax2.set_xticklabels(categories, rotation=45, ha='right')
ax2.set_ylim(0, 105)

# Add value labels on bars
for i, (bar, rate) in enumerate(zip(bars, success_rates)):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
             f'{rate:.1f}%', ha='center', va='bottom')

plt.tight_layout()

# Save the summary plot
plot_path = Path("/home/jovyan/work/outputs/figures/test_summary.png")
plt.savefig(plot_path, dpi=150, bbox_inches='tight')
plt.show()

print(f"\n💾 Summary plot saved to: {plot_path}")

## 7. Save Test Results

Save all test results and scripts to the appropriate directories:

In [ ]:
# Save detailed test results to JSON
timestamp_str = datetime.now().strftime("%Y%m%d_%H%M%S")
results_file = Path(TEST_CONFIG["results_dir"]) / f"integration_test_results_{timestamp_str}.json"

# Save JSON results
with open(results_file, 'w') as f:
    json.dump(summary, f, indent=2, default=str)

print(f"💾 Test results saved to: {results_file}")

# Create a human-readable test report
report_file = Path(TEST_CONFIG["results_dir"]) / f"test_report_{timestamp_str}.txt"

with open(report_file, 'w') as f:
    f.write("DOCKER COMPOSE JUPYTER LAB INTEGRATION TEST REPORT\n")
    f.write("=" * 60 + "\n\n")
    f.write(f"Test Run Timestamp: {summary['test_run_info']['timestamp']}\n")
    f.write(f"Total Tests: {summary['test_run_info']['total_tests']}\n")
    f.write(f"Passed: {summary['test_run_info']['passed']}\n")
    f.write(f"Failed: {summary['test_run_info']['failed']}\n") 
    f.write(f"Warnings: {summary['test_run_info']['warnings']}\n")
    f.write(f"Success Rate: {summary['test_run_info']['success_rate']:.1f}%\n\n")
    
    f.write("CATEGORY BREAKDOWN\n")
    f.write("-" * 30 + "\n")
    for category, results in summary['category_breakdown'].items():
        f.write(f"{category}: {results['passed']}/{results['total']} ({results['success_rate']:.1f}%)\n")
    
    f.write(f"\nDETAILED TEST RESULTS\n")
    f.write("-" * 30 + "\n")
    for result in test_results:
        f.write(f"Test {result['test_id']}: {result['test_name']} - {result['status']}\n")
        if result['message']:
            f.write(f"  Message: {result['message']}\n")
        if result['details']:
            f.write(f"  Details: {json.dumps(result['details'], indent=4)}\n")
        f.write("\n")

print(f"📄 Human-readable report saved to: {report_file}")

# Create a test script template for future use
test_script_template = '''#!/usr/bin/env python3
"""
Docker Compose Jupyter Lab Test Script Template
Generated automatically from integration test notebook
"""

import subprocess
import json
from datetime import datetime
from pathlib import Path

def run_docker_compose_tests():
    """Run basic Docker Compose validation tests"""
    
    results = []
    
    # Test 1: Check Docker Compose configuration
    try:
        result = subprocess.run(['docker', 'compose', 'config'], 
                              capture_output=True, text=True, timeout=30)
        if result.returncode == 0:
            results.append({"test": "compose_config", "status": "PASS"})
        else:
            results.append({"test": "compose_config", "status": "FAIL", "error": result.stderr})
    except Exception as e:
        results.append({"test": "compose_config", "status": "FAIL", "error": str(e)})
    
    # Test 2: Check if services are running
    try:
        result = subprocess.run(['docker', 'compose', 'ps'], 
                              capture_output=True, text=True, timeout=30)
        if result.returncode == 0 and "jupyterlab-datascience" in result.stdout:
            results.append({"test": "service_status", "status": "PASS"})
        else:
            results.append({"test": "service_status", "status": "FAIL", "output": result.stdout})
    except Exception as e:
        results.append({"test": "service_status", "status": "FAIL", "error": str(e)})
    
    return results

if __name__ == "__main__":
    print("Running Docker Compose tests...")
    results = run_docker_compose_tests()
    
    for result in results:
        status_icon = "✅" if result["status"] == "PASS" else "❌"
        print(f"{status_icon} {result['test']}: {result['status']}")
    
    # Save results
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    results_file = Path("results") / f"quick_test_{timestamp}.json"
    results_file.parent.mkdir(exist_ok=True)
    
    with open(results_file, 'w') as f:
        json.dump(results, f, indent=2)
    
    print(f"Results saved to: {results_file}")
'''

script_file = Path(TEST_CONFIG["project_root"]) / "scripts" / "tests" / "docker_compose_validator.py"
with open(script_file, 'w') as f:
    f.write(test_script_template)

print(f"🔧 Test script template saved to: {script_file}")

# Display final summary
print(f"\n🎉 INTEGRATION TEST COMPLETE!")
print(f"📁 All files saved to: {Path(TEST_CONFIG['results_dir'])}")
print(f"📊 Overall Success Rate: {summary['test_run_info']['success_rate']:.1f}%")

if summary['test_run_info']['success_rate'] == 100:
    print("✅ All tests passed! Your Docker Compose Jupyter Lab setup is working perfectly!")
elif summary['test_run_info']['success_rate'] >= 80:
    print("⚠️ Most tests passed! Minor issues detected - check the detailed results.")
else:
    print("❌ Several tests failed! Review the detailed results and fix the issues.")

# List all generated files
print(f"\n📄 Generated Files:")
generated_files = [
    results_file,
    report_file, 
    script_file,
    "/home/jovyan/work/outputs/figures/test_summary.png"
]

for file_path in generated_files:
    if Path(file_path).exists():
        print(f"   ✅ {file_path}")
    else:
        print(f"   ❌ {file_path} (not found)")

print(f"\n🏁 Integration testing completed at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")